In [1]:
import pandas as pd
import numpy as np
import csv,sqlite3
%load_ext sql

In [2]:
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
%sql sqlite:///my_data1.db

In [7]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv')
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [8]:
%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [11]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [13]:
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


## Task 1
### Display the names of the unique launch sites in the space mission

In [15]:
%sql SELECT DISTINCT Launch_Site FROM SPACEXTBL

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


## Task 2
### Display 5 records where launch sites begin with the string 'CCA'

In [17]:
%sql SELECT * FROM SPACEXTBL WHERE Launch_Site LIKE 'CCA%' LIMIT 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


## Task 3
### Display the total payload mass carried by boosters launched by NASA (CRS)

In [19]:
%sql SELECT SUM(PAYLOAD_MASS__KG_) AS TOTAL_MASS_KG FROM SPACEXTBL WHERE Customer = 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


TOTAL_MASS_KG
45596


## Task 4
### Display average payload mass carried by booster version F9 v1.1

In [21]:
%sql SELECT AVG(PAYLOAD_MASS__KG_) AS AVG_MASS_KG FROM SPACEXTBL WHERE Booster_Version = 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


AVG_MASS_KG
2928.4


## Task 5
### List the date when the first succesful landing outcome in ground pad was acheived.

In [23]:
%sql SELECT MIN(Date) AS FIRST_SUCCESS_LAND FROM SPACEXTBL WHERE Landing_Outcome == 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


FIRST_SUCCESS_LAND
2015-12-22


In [25]:
# ANOTHER WAY TO WRITE SAME SCRIPT
%sql SELECT Date FROM SPACEXTBL WHERE Landing_Outcome == 'Success (ground pad)' ORDER BY Date ASC LIMIT 1

 * sqlite:///my_data1.db
Done.


Date
2015-12-22


## Task 6
### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [27]:
%sql SELECT Booster_Version FROM SPACEXTBL WHERE PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000 AND Landing_Outcome = 'Success (drone ship)'

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


## Task 7
### List the total number of successful and failure mission outcomes

In [29]:
%sql SELECT Landing_Outcome, COUNT(*) AS TOTAL_OUTCOME FROM SPACEXTBL GROUP BY Landing_Outcome

 * sqlite:///my_data1.db
Done.


Landing_Outcome,TOTAL_OUTCOME
Controlled (ocean),5
Failure,3
Failure (drone ship),5
Failure (parachute),2
No attempt,21
No attempt,1
Precluded (drone ship),1
Success,38
Success (drone ship),14
Success (ground pad),9


## Task 8
### List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [33]:
%sql SELECT Booster_Version FROM SPACEXTBL WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


## Task 9
### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [38]:
%%sql SELECT 
    CASE SUBSTR(Date, 6, 2)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END AS Month_Name,
    Booster_Version,
    Launch_Site
FROM SPACEXTBL
WHERE Landing_Outcome = 'Failure (drone ship)'
AND SUBSTR(Date, 1, 4) = '2015';


 * sqlite:///my_data1.db
Done.


Month_Name,Booster_Version,Launch_Site
January,F9 v1.1 B1012,CCAFS LC-40
April,F9 v1.1 B1015,CCAFS LC-40


## Task 10
### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [41]:
%%sql SELECT Landing_Outcome, COUNT(*) AS Outcome_count FROM SPACEXTBL WHERE DATE BETWEEN '2010-06-04' 
AND '2017-03-20' GROUP BY Landing_Outcome ORDER BY Outcome_Count DESC

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Outcome_count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
